In [1]:
import requests
from bs4 import BeautifulSoup
from mpi4py import MPI
from googletrans import Translator
from fpdf import FPDF

In [3]:
# Get the number of processes
comm = MPI.COMM_WORLD
size = comm.Get_size()

In [4]:
# Connect to the website
resp = requests.get("https://www.vinmec.com/vi/benh/")
soup = BeautifulSoup(resp.text, 'html.parser')

In [5]:
# Get the link extentions and divide them into processes
uls = soup.find_all('ul', class_='collapsible-target')
lis = [li for ul in uls for li in ul.find_all('li')]
link_extensions = [li.find('a')['href'] for li in lis if li.find('a') is not None]
# splited_link_extentions = [link_extentions[i::size] for i in range(size)]
# comm.Scatter(splited_link_extentions, link_extentions, root=0)

In [22]:
# Get and process the content of per link
# for link in link_extentions:
link = link_extensions[0]
link = "https://www.vinmec.com" + link
resp = requests.get(link)
soup = BeautifulSoup(resp.text, 'html.parser')
content = soup.find('div', class_='content')

translator = Translator()

pdf=FPDF()
pdf.add_page()
pdf.add_font('DejaVu', '', '../font/DejaVuSansCondensed.ttf', uni=True)
pdf.set_font('DejaVu', '', 14)

for element in content.find_all('p'):
    text = element.get_text()
    if text == '\xa0':
        break
    pdf.write(8, translator.translate(text, src='vi', dest='en').text)
    pdf.ln()
    
pdf.output('../data/a.pdf','F')

''